# Exploring Country Wise MBTI Personalities

## Approach
Look at different traits (specifically: I, N, F, P) and MBTI Types and their variation across countries. 

## Bit of background 
MBTI is a (one of many) way to understand psychology behind how people perceive the world & make decisions. It is set of 4 traits (4 boolean variables) assigned to a person based on a test.

### Traits
The (Fantastic) Four Dichotomies
1. **I vs E**: Introversion vs Extraversion - Attitude - This talks about the general attitude of the person towards external world 
2. **S vs N**: Sensing vs Intuition - Perceiving functions - This talks about the way the person gathers information 
3. **T vs F**: Thinking vs Feeling - Judging functions - This talks about the way the person makes decisions 
4. **J vs P**: Judging vs Perception - Lifestyle preference - This talks about the way the person which of the above two does the person prefer when dealing with external world 

One combination of the four traits makes one MBTI Type. So, total number of MBTI Types are (How many?) 16 (You guessed right!). 

Intrigued?! You can take the personality test online to know what type you belong to! (e.g., at 16personalities.com).

Or, you can put all the text you have ever written through the ML model I made which predicts your MBTI type from text (How cool is that!). Check it out here: [Predicting MBTI Type](https://www.kaggle.com/rajjain/predict-mbti-type-from-text-using-tpu) & [Predicting MBTI Traits](https://www.kaggle.com/rajjain/predict-mbti-traits-from-text-using-tpu)

(More info on this [Myers–Briggs Type Indicator Wiki Page](https://en.wikipedia.org/wiki/Myers%E2%80%93Briggs_Type_Indicator). Highly encourage you to give this a look-see for usage & criticism of the method. Also, [16personalities](https://www.16personalities.com/) for a better UX than Wikipedia :P)

# Dataset Explanation

## Extra trait (Wait, what?!)

There is a new trait that is used in addition to the original four (Fantastic Five now?!), which is, **A vs T**: Assertive vs Turbulent - Identity - This talks about how confident we are in our abilities and decisions! 

For the purpose of this notebook, I will focus on the original four (Whew!) 

## What does the dataset give us?!

The dataset gives us for each country, what is the percentage of the population belonging to each MBTI Type. We will sum respective information to get the distribution at trait level / original MBTI type for each country. 

# Detailed Approach

* We will use Choropleth maps to visualise the data effectively. We will look at different configurations of the map for different trait and make a conclusion on which was the best way. We will use plotly for this. One of the best visualisation libraries for Python!
* We will also visualise at MBTI types variation w.r.t each country. Through sliders and animation! Again using Plotly!

In [ ]:
import numpy 
import pandas
import requests
import plotly.express as px
import plotly.graph_objects as go

fav_country = 'India'

In [ ]:
type_info = pandas.read_csv('/kaggle/input/mbtitypes-full/types.csv', index_col='Type')
df = pandas.read_csv('/kaggle/input/mbtitypes-full/countries.csv', index_col='Country')
pandas.options.display.max_columns = 32
df.loc[[fav_country]] * 100

In [ ]:
types = df.columns

one_slice = ['I', 'N', 'F', 'P']  # One slice of the indicators

traits_df = pandas.DataFrame(index=df.index, columns=one_slice)

for ind, trait in enumerate(one_slice):
    columns_to_sum = list(filter(lambda x: x[ind] == trait, types))
    traits_df[trait] = df[columns_to_sum].sum(axis=1) * 100

traits_df.loc[[fav_country]][one_slice]

In [ ]:
for trait, trait_name, projection_type in zip(one_slice, ['Introversion', 'Intuitive', 'Feeling', 'Perceptive'], 
                                              ['equirectangular', 'orthographic', 'natural earth', 'sinusoidal']):
    fig = go.Figure(data=go.Choropleth(
        locations = traits_df.index,
        z = traits_df[trait],
        locationmode='country names',
        reversescale=True,
        colorbar_title = '{} %'.format(trait_name),
        colorbar_ticksuffix = '%',
    ))
    fig.update_layout(
        title={'text': 'Countries with {} % - {}'.format(trait_name, projection_type), 'y': 0.9, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top'},
        geo=dict(projection_type=projection_type))
    fig.show()

## Have fun zooming / moving the above plots

### It makes sense to use "natural earth" or "equirectangular" for this visualisation. All info is present at one glance. 

But you have to agree that "orthographic" is plain fun!

In [ ]:
original_types = sorted(list(set(map(lambda x: x.split('-')[0], types))))
original_types_df = pandas.DataFrame(index=df.index, columns=original_types)

for otype in original_types:
    columns_to_sum = list(filter(lambda x: otype in x, types))
    original_types_df[otype] = df[columns_to_sum].sum(axis=1) * 100

max_type = original_types_df.idxmax(axis=1)

In [ ]:
fav_type = max_type[fav_country]

layout = {
    "xaxis": {"title": "MBTI Type"},
    "yaxis": {"title": "Percentage of Population", "range": [0, 21]},
    "title_text": fav_country + ' - ' + type_info.loc[fav_type]['Nickname'] + ' - ' + type_info.loc[fav_type]['Description'],
    "updatemenus": [{
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 2000, "redraw": True}, "fromcurrent": True, 
                                "transition": {"duration": 300, "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "showactive": False,
        "pad": {"r": 10, "t": 87},
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }]
}

sliders_dict = {"active": original_types_df.index.tolist().index(fav_country), 
    "yanchor": "top", "xanchor": "left", "len": 0.9, "x": 0.1, "y": 0, "steps": [], "pad": {"b": 10, "t": 50},
    "currentvalue": {"font": {"size": 20}, "prefix": "Country: ", "visible": True, "xanchor": "right"},
    "transition": {"duration": 300, "easing": "cubic-in-out"},
}

frames = []
for country in original_types_df.index:    
    bar = go.Bar(x=original_types, y=original_types_df.loc[country].tolist())
    country_max = max_type[country]
    title = country + ' - ' + type_info.loc[country_max]['Nickname'] + ' - ' + type_info.loc[country_max]['Description']
    frame = go.Frame(data=[bar], name=country, layout=go.Layout(title_text=title))
    frames.append(frame)
    
    slider_step = {
        "args": [
            [country],
            {"frame": {"duration": 300, "redraw": True}, "mode": "immediate", "transition": {"duration": 300}}
        ],
        "label": country,
        "method": "animate"
    }
    sliders_dict["steps"].append(slider_step)

layout['sliders'] = [sliders_dict]
bar = go.Bar(x=original_types, y=original_types_df.loc[fav_country].tolist())  # Initial data

fig = go.Figure(data=[bar], frames=frames, layout=layout)
fig.show()

### Wow! My country has lot of people who are Campaigners! Lot of smiling people in India! Make sure you visit :) 

All this was possible by learning and getting inspired from multiple sources. "Standing on the shoulders of Giants :)"